# Problem Statement:

Tennis Australia Open is trying to better automate how tennis points get categorized into three outcomes –

* `Winner` – the point winning player hits a shot that is not touched by the opponent
* `Forced error` – the point winning player hits a shot that causes the opponent to not be able to return it, i.e. a good shot that is hard to handle
* `Unforced error` – the player attempting to return the ball makes an error on an otherwise normal looking rally shot

## Dataset Description:

The dataset includes point outcomes of rallies only (where the number of shots hit exceeds two, which represents the serve and return). All points were played at a past Australian Open.

## Atribute description

| Variable | Description| Value Range |
| :- | -: | :-: |
rally | The number of shots in the point counting serves and point-ending shot | An integer from 1, 2, 3...
| serve | A number indicating whether the point was played on a first or second serve.  | 1 = First, 2 = Second
| hitpoint | Shot category for point-ending shot | F = Forehand, B = Backhand, V = Volley, U = Unknown
| speed | Speed of point-ending shot | Continuous (m/s)
| net.clearance | Distance above the net as point-ending shot passed the net | Continuous (cm) distance above net. Can be negative if shot did not pass above the net.
| distance.from.sideline | Lateral distance of the point-ending shot bounce from the nearest singles sideline. | Perpendicular distance in meters (always positive even if out)
| depth | Distance of the point-ending shot bounce from the baseline | Perpendicular distance in meters
(always positive even if out)
| outside.sideline | Logical indicator of whether point-ending shot landed outside of the in-play singles sideline | TRUE, FALSE
| outside.baseline | Logical indicator of whether point-ending shot landed beyond the in-play baseline | TRUE, FALSE
| player.distance.travelled | Distance player who made the point-ending shot travelled between the impact of the penultimate shot and the impact of the point-ending shot | Euclidean distance in meters
| player.impact.depth | Distance of player who made point-ending shot from the net at the time the point-ending shot was made | Perpendicular distance along the length of court from net in meters
| player.impact.distance.from.center | Distance of player who made point-ending shot from the center line at the time the point-ending shot was made | Perpendicular distance from the center line in meters
| player.depth | Distance of player who made point-ending shot from the net at the time the penultimate shot was made | Perpendicular distance along the length of court from net in meters
| player.distance.from.center | Distance of player who made point-ending shot from the center line at the time the penultimate shot was made | Perpendicular distance from the center line in meters
| opponent.depth | Distance of opponent from the net at the time the at the time the penultimate shot was made | Perpendicular distance along the length of court from net in meters
| opponent.distance.from.center | Distance of opponent from the center line at the time the penultimate shot was made | Perpendicular distance from the center line in meters
| same.side | Logical indicator if both player and opponent were positioned on the same side of the center line (ad or deuce court) at the time the penultimate shot was made | TRUE, FALSE
| previous.speed | Speed of penultimate shot | Continuous (m/s)
| previous.net.clearance | Distance above the net as penultimate shot passed the net | Continuous (cm) distance above net. Can be negative if shot did not pass above the net.
| previous.distance.from.sideline | Lateral distance of the penultimate  shot bounce from the nearest singles sideline. | Perpendicular distance in meters (always positive even if out)
| previous.depth | Distance of the penultimate shot bounce from the baseline | Perpendicular distance in meters
(always positive even if out)
| previous.hitpoint | Shot category for penultimate shot | F = Forehand, B = Backhand, V = Volley, U = Unknown
| previous.time.to.net | Time for penultimate shot to be hit and pass the net | Continuous number in seconds
| server.is.impact.player | Logical if player who made point-ending shot was the server of the point | TRUE, FALSE
| outcome | Target variable, character with three categories indicating the type of shot that ended the point  | W (Winner), FE (Forced Error), UE (Unforced Error)
| id | A 10-character unique identifier for the point | Character

## Let's find the well fitted model for the dataset

# Import libraries

In [ ]:
#Importing the necessary libraries and packages for the whole project
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report,confusion_matrix

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier

from xgboost import XGBClassifier

from sklearn.ensemble import VotingClassifier
#from vecstack import stacking

from sklearn.model_selection import GridSearchCV

#Mounting thedrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load data

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/ML/stacking/tennis.csv")

# Data Understanding

## Number of records and columns

In [ ]:
data.shape

(8001, 27)

## See the first five records

In [ ]:
data.head().T

,0,1,2,3,4
rally,4,4,23,9,4
serve,1,2,1,1,1
hitpoint,B,B,B,F,B
speed,35.515042,33.38264,22.31669,36.837309,35.544208
net.clearance,-0.021725,1.114202,-0.254046,0.766694,0.116162
distance.from.sideline,3.474766,2.540801,3.533166,0.586885,0.918725
depth,6.797621,2.608708,9.435749,3.34218,5.499119
outside.sideline,False,False,False,True,False
outside.baseline,False,True,False,False,False
player.distance.travelled,1.46757,2.311931,3.903728,0.583745,2.333456


In [ ]:
data['rally'].min() #The number of shots in the point counting serves and point-ending shot	has minimun 3 shots.

3

## Different classes in Outcome variable

In [ ]:
data.outcome.value_counts()

UE    3501
W     2682
FE    1818
Name: outcome, dtype: int64

In [ ]:
data.outcome.value_counts(normalize= True)*100

UE    43.75703
W     33.52081
FE    22.72216
Name: outcome, dtype: float64

## Display data type of each variable

In [ ]:
data.dtypes

rally                                   int64
serve                                   int64
hitpoint                               object
speed                                 float64
net.clearance                         float64
distance.from.sideline                float64
depth                                 float64
outside.sideline                         bool
outside.baseline                         bool
player.distance.travelled             float64
player.impact.depth                   float64
player.impact.distance.from.center    float64
player.depth                          float64
player.distance.from.center           float64
previous.speed                        float64
previous.net.clearance                float64
previous.distance.from.sideline       float64
previous.depth                        float64
opponent.depth                        float64
opponent.distance.from.center         float64
same.side                                bool
previous.hitpoint                 

## Identifying categorical attributes

In [ ]:
categorical_list = ["hitpoint","outside.sideline",
                    "outside.baseline","same.side",
                    "previous.hitpoint",
                    "server.is.impact.player",
                    "gender","outcome"]

## Converting to appropriate datatype

In [ ]:
data[categorical_list] = data[categorical_list].astype("category")

## Display data type of each variable after conversion

In [ ]:
data.dtypes

rally                                    int64
serve                                    int64
hitpoint                              category
speed                                  float64
net.clearance                          float64
distance.from.sideline                 float64
depth                                  float64
outside.sideline                      category
outside.baseline                      category
player.distance.travelled              float64
player.impact.depth                    float64
player.impact.distance.from.center     float64
player.depth                           float64
player.distance.from.center            float64
previous.speed                         float64
previous.net.clearance                 float64
previous.distance.from.sideline        float64
previous.depth                         float64
opponent.depth                         float64
opponent.distance.from.center          float64
same.side                             category
previous.hitp

## Dropping ID column and checking the length of columns

In [ ]:
data.shape

(8001, 27)

In [ ]:
len(data['ID'].unique()) #For every shot there is a unique Id is given so there is no use of the column 'ID'.


8001

In [ ]:
data.drop(["ID"], axis=1, inplace=True) # Dropping the 'ID' column.
len(data.columns)

26

## Display summary statistics

In [ ]:
data.describe() # Statistical description of numerical variables

,rally,serve,speed,net.clearance,distance.from.sideline,depth,player.distance.travelled,player.impact.depth,player.impact.distance.from.center,player.depth,player.distance.from.center,previous.speed,previous.net.clearance,previous.distance.from.sideline,previous.depth,opponent.depth,opponent.distance.from.center,previous.time.to.net
count,8001.000000,8001.000000,8001.000000,8001.000000,8001.000000,8001.000000,8001.000000,8001.000000,8001.000000,8001.000000,8001.000000,8001.000000,8001.000000,8001.000000,8001.000000,8001.000000,8001.000000,8001.000000
mean,5.966004,1.398700,30.806938,0.629658,1.467630,4.421146,2.690463,11.899694,1.919544,12.253954,1.213795,28.763676,0.821562,2.193420,4.218717,12.616810,2.367952,0.549988
std,3.548182,0.489661,7.298917,0.982504,1.108697,3.144965,1.713136,2.788231,1.205449,2.039085,0.964364,6.477470,0.674663,1.038942,2.052946,2.075401,1.313927,0.186788
min,3.000000,1.000000,5.176078,-0.998184,0.000497,0.003135,0.000000,2.156000,0.000200,1.389800,0.000400,8.449117,0.028865,0.000164,0.000467,2.161200,0.000200,0.003201
25%,3.000000,1.000000,26.770290,-0.027092,0.539500,1.641161,1.444233,11.221400,0.942400,11.374200,0.551800,24.033218,0.404815,1.354458,2.733674,12.082400,1.352200,0.432164
50%,5.000000,1.000000,32.417690,0.445870,1.210847,3.860266,2.360894,12.691800,1.829400,12.551600,0.983800,29.793417,0.658382,2.168822,4.126864,12.901600,2.332000,0.507559
75%,7.000000,2.000000,35.681431,0.970844,2.215955,7.029345,3.565853,13.553000,2.745200,13.498000,1.596600,33.581003,1.021397,3.022677,5.595515,13.712800,3.259000,0.624135
max,38.000000,2.000000,55.052795,12.815893,7.569757,11.886069,14.480546,18.125600,7.746200,18.745800,9.352600,54.207506,6.730275,4.114361,9.997963,20.211000,6.852600,1.635257


In [ ]:
data.describe(include=['category']) #Statistical description of categorical variables

,hitpoint,outside.sideline,outside.baseline,same.side,previous.hitpoint,server.is.impact.player,outcome,gender
count,8001,8001,8001,8001,8001,8001,8001,8001
unique,4,2,2,2,4,2,3,2
top,F,False,False,False,F,True,UE,mens
freq,4402,6500,6380,6036,3684,4670,3501,4005


## Check the distribution of all categorical attributes

In [ ]:
for i in categorical_list:
    print('--------'+i+'---------\n')
    print(data[i].value_counts(normalize=True)*100)
    print('\n')

--------hitpoint---------

F    55.018123
B    37.920260
U     5.361830
V     1.699788
Name: hitpoint, dtype: float64


--------outside.sideline---------

False    81.239845
True     18.760155
Name: outside.sideline, dtype: float64


--------outside.baseline---------

False    79.740032
True     20.259968
Name: outside.baseline, dtype: float64


--------same.side---------

False    75.44057
True     24.55943
Name: same.side, dtype: float64


--------previous.hitpoint---------

F    46.044244
B    40.969879
V     8.998875
U     3.987002
Name: previous.hitpoint, dtype: float64


--------server.is.impact.player---------

True     58.367704
False    41.632296
Name: server.is.impact.player, dtype: float64


--------gender---------

mens      50.056243
womens    49.943757
Name: gender, dtype: float64


--------outcome---------

UE    43.75703
W     33.52081
FE    22.72216
Name: outcome, dtype: float64




## Checking for null values

In [ ]:
data.isnull().sum()

rally                                 0
serve                                 0
hitpoint                              0
speed                                 0
net.clearance                         0
distance.from.sideline                0
depth                                 0
outside.sideline                      0
outside.baseline                      0
player.distance.travelled             0
player.impact.depth                   0
player.impact.distance.from.center    0
player.depth                          0
player.distance.from.center           0
previous.speed                        0
previous.net.clearance                0
previous.distance.from.sideline       0
previous.depth                        0
opponent.depth                        0
opponent.distance.from.center         0
same.side                             0
previous.hitpoint                     0
previous.time.to.net                  0
server.is.impact.player               0
outcome                               0


* No null values are present in the data

# Divide the data into train and test

In [ ]:
y = data["outcome"]
X = data.drop('outcome', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=123, stratify=y)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5600, 25)
(2401, 25)
(5600,)
(2401,)


# Preprocessing

## Display all the columns

In [ ]:
data.columns

Index(['rally', 'serve', 'hitpoint', 'speed', 'net.clearance',
       'distance.from.sideline', 'depth', 'outside.sideline',
       'outside.baseline', 'player.distance.travelled', 'player.impact.depth',
       'player.impact.distance.from.center', 'player.depth',
       'player.distance.from.center', 'previous.speed',
       'previous.net.clearance', 'previous.distance.from.sideline',
       'previous.depth', 'opponent.depth', 'opponent.distance.from.center',
       'same.side', 'previous.hitpoint', 'previous.time.to.net',
       'server.is.impact.player', 'outcome', 'gender'],
      dtype='object')

## Creating a list of numerical attributes and categorical list

In [ ]:
numeric_list = ['rally', 'serve', 'speed', 'net.clearance',
                'distance.from.sideline', 'depth',
                'player.distance.travelled', 'player.impact.depth',
                'player.impact.distance.from.center',
                'player.depth', 'player.distance.from.center',
                'previous.speed', 'previous.net.clearance',
                'previous.distance.from.sideline', 'previous.depth',
                'opponent.depth', 'opponent.distance.from.center',
                'previous.time.to.net']

categorical_list = ["hitpoint", "outside.sideline", "outside.baseline", "same.side",
                    "previous.hitpoint", "server.is.impact.player", "gender"]

In [ ]:
len(numeric_list)

18

In [ ]:
len(categorical_list)

7

## LabelEncoder  : Target Attributes

In [ ]:
y_train.value_counts()

UE    2450
W     1877
FE    1273
Name: outcome, dtype: int64

In [ ]:
le = LabelEncoder()

le.fit(y_train)

y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [ ]:
print(le.classes_)

['FE' 'UE' 'W']


In [ ]:
print(le.inverse_transform([0, 1, 2]))

['FE' 'UE' 'W']


In [ ]:
np.unique(y_train, return_counts=True)
# 0 - Forced Error, 1 - Unforced Error, 2 - Winner

(array([0, 1, 2]), array([1273, 2450, 1877]))

In [ ]:
np.unique(y_test, return_counts=True)

(array([0, 1, 2]), array([ 545, 1051,  805]))

## Standard Scaler : Independent Numberic Attributes

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train[numeric_list])

X_train_num = pd.DataFrame(scaler.transform(X_train[numeric_list]), columns=numeric_list)
X_test_num = pd.DataFrame(scaler.transform(X_test[numeric_list]), columns=numeric_list)

In [ ]:
X_train_num.head()

,rally,serve,speed,net.clearance,distance.from.sideline,depth,player.distance.travelled,player.impact.depth,player.impact.distance.from.center,player.depth,player.distance.from.center,previous.speed,previous.net.clearance,previous.distance.from.sideline,previous.depth,opponent.depth,opponent.distance.from.center,previous.time.to.net
0,2.206016,1.228398,1.121552,-0.730696,-0.175183,0.988152,-0.829916,1.120228,0.808667,1.526350,0.482096,0.548899,0.520014,1.276148,-0.584783,0.454714,-0.368646,-0.231291
1,4.136322,-0.814069,1.170424,0.163662,-0.862930,-1.148001,0.014953,1.102595,0.362499,0.702730,-1.249078,0.196423,0.152115,-1.020791,-1.205634,-0.887901,0.331354,-0.808893
2,-0.000049,1.228398,0.182863,0.532959,-1.015778,-1.404791,0.531566,0.340187,0.245007,0.377467,0.154493,1.550284,-0.843765,-0.169719,-0.079680,1.267120,-0.574296,-0.513104
3,-0.827324,-0.814069,0.245147,-0.189687,-0.771740,-0.467465,-0.234643,0.107888,0.884055,-0.367821,-0.410447,0.390784,-0.559132,-0.237628,0.418002,-0.342972,1.316556,-0.546510
4,-0.827324,-0.814069,-0.396162,-0.647252,1.065523,0.586853,-1.105658,-0.557186,-0.463438,-1.017955,-0.630639,0.824545,-0.876408,0.551589,0.361632,-0.144350,1.534831,-0.648883


## OneHotEncoder : Independent Categorical Attributes

In [ ]:
ohe = OneHotEncoder()

ohe.fit(X_train[categorical_list])

columns_ohe = list(ohe.get_feature_names_out(categorical_list))
print(columns_ohe)

['hitpoint_B', 'hitpoint_F', 'hitpoint_U', 'hitpoint_V', 'outside.sideline_0.0', 'outside.sideline_1.0', 'outside.baseline_0.0', 'outside.baseline_1.0', 'same.side_0.0', 'same.side_1.0', 'previous.hitpoint_B', 'previous.hitpoint_F', 'previous.hitpoint_U', 'previous.hitpoint_V', 'server.is.impact.player_0.0', 'server.is.impact.player_1.0', 'gender_mens', 'gender_womens']


In [ ]:
X_train_cat = ohe.transform(X_train[categorical_list])
X_test_cat  = ohe.transform(X_test[categorical_list])

In [ ]:
X_train_cat = pd.DataFrame(X_train_cat.todense(), columns=columns_ohe)
X_test_cat  = pd.DataFrame(X_test_cat.todense(), columns=columns_ohe)

NameError: ignored

In [ ]:
X_train_cat.head()

,hitpoint_B,hitpoint_F,hitpoint_U,hitpoint_V,outside.sideline_0.0,outside.sideline_1.0,outside.baseline_0.0,outside.baseline_1.0,same.side_0.0,same.side_1.0,previous.hitpoint_B,previous.hitpoint_F,previous.hitpoint_U,previous.hitpoint_V,server.is.impact.player_0.0,server.is.impact.player_1.0,gender_mens,gender_womens
0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


## Concatenate

Concatenating the standardized numerical variables and encoded catergorical variable as single dataframe.

In [ ]:
X_train = pd.concat([X_train_num, X_train_cat], axis=1)
X_test = pd.concat([X_test_num, X_test_cat], axis=1)

In [ ]:
print(X_train.shape, X_test.shape)

(5600, 36) (2401, 36)


In [ ]:
X_train.head()

,rally,serve,speed,net.clearance,distance.from.sideline,depth,player.distance.travelled,player.impact.depth,player.impact.distance.from.center,player.depth,...,same.side_0.0,same.side_1.0,previous.hitpoint_B,previous.hitpoint_F,previous.hitpoint_U,previous.hitpoint_V,server.is.impact.player_0.0,server.is.impact.player_1.0,gender_mens,gender_womens
0,2.206016,1.228398,1.121552,-0.730696,-0.175183,0.988152,-0.829916,1.120228,0.808667,1.526350,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,4.136322,-0.814069,1.170424,0.163662,-0.862930,-1.148001,0.014953,1.102595,0.362499,0.702730,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,-0.000049,1.228398,0.182863,0.532959,-1.015778,-1.404791,0.531566,0.340187,0.245007,0.377467,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,-0.827324,-0.814069,0.245147,-0.189687,-0.771740,-0.467465,-0.234643,0.107888,0.884055,-0.367821,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
4,-0.827324,-0.814069,-0.396162,-0.647252,1.065523,0.586853,-1.105658,-0.557186,-0.463438,-1.017955,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [ ]:
X_test.head()

,rally,serve,speed,net.clearance,distance.from.sideline,depth,player.distance.travelled,player.impact.depth,player.impact.distance.from.center,player.depth,...,same.side_0.0,same.side_1.0,previous.hitpoint_B,previous.hitpoint_F,previous.hitpoint_U,previous.hitpoint_V,server.is.impact.player_0.0,server.is.impact.player_1.0,gender_mens,gender_womens
0,-0.000049,-0.814069,1.915802,-0.209410,-0.993875,-1.313488,-1.414688,0.872303,0.124021,1.063491,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,-0.827324,-0.814069,0.637883,-0.521776,0.027012,0.172605,-1.042676,-0.261741,-1.576280,-0.420140,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,-0.827324,1.228398,1.285836,-0.255016,-0.385628,0.103080,-0.075567,0.665664,-1.426336,-0.452510,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,-0.827324,1.228398,0.191722,-0.185242,-0.566945,-0.193971,-0.799147,0.413152,-0.663473,-0.165681,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,-0.827324,-0.814069,-0.623300,-0.431102,-0.535634,0.740060,-1.082177,0.277543,-0.569446,-0.176144,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


# Error Metrics

## Function to calculate accuracy, recall, precision and F1 score

In [ ]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score',
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")

    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description,
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ),
                                       ignore_index=True)

    return(dataframe)

## Function for Classification Report

In [ ]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

# Model Building

## Decision Trees

In [ ]:
clf_dt = DecisionTreeClassifier()

In [ ]:
clf_dt.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [ ]:
confusion_matrix(y_train, y_pred_train) # Confusion_matrix for the decision tree classifier

array([[1273,    0,    0],
       [   0, 2450,    0],
       [   0,    0, 1877]])

In [ ]:
classifcation_report_train_test(y_train, y_pred_train, y_test, y_pred_test)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000      1273
           1     1.0000    1.0000    1.0000      2450
           2     1.0000    1.0000    1.0000      1877

    accuracy                         1.0000      5600
   macro avg     1.0000    1.0000    1.0000      5600
weighted avg     1.0000    1.0000    1.0000      5600


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.6382    0.6312    0.6347       545
           1     0.7985    0.7992    0.7989      1051
           2     0.9025    0.9081    0.9053       805

    accuracy                         0.7976      2401
   macro avg     0.7797    0.7795    0.7796      2401
weighted avg     0.7970    0.7976    0.7973      2401



In [ ]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-63-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,1.0,1.0,1.0,1.0,0.797584,0.797584,0.796967,0.797268


* By observing the performance metrics it is clearly observed that the decision tree model is overfitted.

## Random Forests

In [ ]:
clf_rf = RandomForestClassifier()

In [ ]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier()

In [ ]:
y_pred_train = clf_rf.predict(X_train)
y_pred_test = clf_rf.predict(X_test)

In [ ]:
confusion_matrix(y_train, y_pred_train) # Confusion matrix for random forest.

array([[1273,    0,    0],
       [   0, 2450,    0],
       [   0,    0, 1877]])

In [ ]:
classifcation_report_train_test(y_train, y_pred_train, y_test, y_pred_test)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000      1273
           1     1.0000    1.0000    1.0000      2450
           2     1.0000    1.0000    1.0000      1877

    accuracy                         1.0000      5600
   macro avg     1.0000    1.0000    1.0000      5600
weighted avg     1.0000    1.0000    1.0000      5600


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.7809    0.7064    0.7418       545
           1     0.8519    0.8754    0.8634      1051
           2     0.9215    0.9478    0.9345       805

    accuracy                         0.8613      2401
   macro avg     0.8514    0.8432    0.8466      2401
weighted avg     0.8591    0.8613    0.8597      2401



In [ ]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "RandomForest", scores)
scores

<ipython-input-63-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,1.0,1.0,1.0,1.0,0.797584,0.797584,0.796967,0.797268
1,RandomForest,1.0,1.0,1.0,1.0,0.861308,0.861308,0.859105,0.859650


* By observing the performance metrics it is observed that random forest model also overfitted

## Build Gradient Boosting Classifier

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train = clf_gbm.predict(X_train)
y_pred_test = clf_gbm.predict(X_test)

In [ ]:
confusion_matrix(y_train, y_pred_train) # confusion matrix for Gradient Boosting Classifier

array([[1047,  198,   28],
       [ 111, 2268,   71],
       [  15,   29, 1833]])

In [ ]:
classifcation_report_train_test(y_train, y_pred_train, y_test, y_pred_test)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           0     0.8926    0.8225    0.8561      1273
           1     0.9090    0.9257    0.9173      2450
           2     0.9488    0.9766    0.9625      1877

    accuracy                         0.9193      5600
   macro avg     0.9168    0.9082    0.9119      5600
weighted avg     0.9186    0.9193    0.9185      5600


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.7859    0.7138    0.7481       545
           1     0.8629    0.8744    0.8686      1051
           2     0.9180    0.9590    0.9380       805

    accuracy                         0.8663      2401
   macro avg     0.8556    0.8491    0.8516      2401
weighted avg     0.8639    0.8663    0.8645      2401



In [ ]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "GBM", scores)
scores

<ipython-input-63-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,1.000000,1.000000,1.000000,1.000000,0.797584,0.797584,0.796967,0.797268
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.861308,0.861308,0.859105,0.859650
2,GBM,0.919286,0.919286,0.918602,0.918517,0.866306,0.866306,0.863876,0.864530


* By observing the performace metrics Gradient Boostig is better than decision tree and random forest

## AdaBoost

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train = clf_adaboost.predict(X_train)
y_pred_test = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train, y_test, y_pred_test)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           0     0.7465    0.6779    0.7106      1273
           1     0.8365    0.8665    0.8512      2450
           2     0.9297    0.9441    0.9368      1877

    accuracy                         0.8496      5600
   macro avg     0.8376    0.8295    0.8329      5600
weighted avg     0.8473    0.8496    0.8480      5600


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.7166    0.6587    0.6864       545
           1     0.8341    0.8516    0.8427      1051
           2     0.9117    0.9366    0.9240       805

    accuracy                         0.8363      2401
   macro avg     0.8208    0.8156    0.8177      2401
weighted avg     0.8335    0.8363    0.8345      2401



In [ ]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "Adaboost", scores)
scores

<ipython-input-63-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,1.000000,1.000000,1.000000,1.000000,0.797584,0.797584,0.796967,0.797268
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.861308,0.861308,0.859105,0.859650
2,GBM,0.919286,0.919286,0.918602,0.918517,0.866306,0.866306,0.863876,0.864530
3,Adaboost,0.849643,0.849643,0.847281,0.847952,0.836318,0.836318,0.833453,0.834513


* By observing the performance metrics we can say that the AdaBoost model fitted good.

## XGBOOST

In [ ]:
clf_xgb = XGBClassifier()

In [ ]:
clf_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [ ]:
y_pred_train = clf_xgb.predict(X_train)
y_pred_test = clf_xgb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train, y_test, y_pred_test)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000      1273
           1     1.0000    1.0000    1.0000      2450
           2     1.0000    1.0000    1.0000      1877

    accuracy                         1.0000      5600
   macro avg     1.0000    1.0000    1.0000      5600
weighted avg     1.0000    1.0000    1.0000      5600


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.7857    0.7468    0.7658       545
           1     0.8757    0.8716    0.8736      1051
           2     0.9164    0.9528    0.9342       805

    accuracy                         0.8705      2401
   macro avg     0.8593    0.8570    0.8579      2401
weighted avg     0.8689    0.8705    0.8695      2401



# Performing grid search for better hyper parameters

In [ ]:
XGB = XGBClassifier(n_jobs=-1)

# Use a grid over parameters of interest
param_grid = {
    'colsample_bytree': np.linspace(0.6, 0.8, 2),
    'n_estimators': [50, 100],
    'max_depth': [5, 6]}

CV_XGB = GridSearchCV(estimator=XGB, param_grid=param_grid, cv=3)

In [ ]:
CV_XGB.fit(X=X_train, y=y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=-1,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             param_grid={'colsample_bytree': array([0.6, 0.8]),
                         'max_depth': [5, 6], 'n_estimators': [50, 100]})

Find best model

In [ ]:
best_xgb_model = CV_XGB.best_estimator_

In [ ]:
print(CV_XGB.best_score_, CV_XGB.best_params_)

0.8728583148048322 {'colsample_bytree': 0.6, 'max_depth': 6, 'n_estimators': 100}


In [ ]:
best_xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [ ]:
y_pred_train = best_xgb_model.predict(X_train)
y_pred_test = best_xgb_model.predict(X_test)

In [ ]:
confusion_matrix(y_train,y_pred_train)

array([[1273,    0,    0],
       [   0, 2450,    0],
       [   0,    0, 1877]])

In [ ]:
classifcation_report_train_test(y_train, y_pred_train, y_test, y_pred_test)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000      1273
           1     1.0000    1.0000    1.0000      2450
           2     1.0000    1.0000    1.0000      1877

    accuracy                         1.0000      5600
   macro avg     1.0000    1.0000    1.0000      5600
weighted avg     1.0000    1.0000    1.0000      5600


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.7964    0.7394    0.7669       545
           1     0.8700    0.8849    0.8774      1051
           2     0.9310    0.9553    0.9430       805

    accuracy                         0.8755      2401
   macro avg     0.8658    0.8599    0.8624      2401
weighted avg     0.8737    0.8755    0.8743      2401



In [ ]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "XGBoost", scores)
scores

<ipython-input-63-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,1.000000,1.000000,1.000000,1.000000,0.797584,0.797584,0.796967,0.797268
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.861308,0.861308,0.859105,0.859650
2,GBM,0.919286,0.919286,0.918602,0.918517,0.866306,0.866306,0.863876,0.864530
3,Adaboost,0.849643,0.849643,0.847281,0.847952,0.836318,0.836318,0.833453,0.834513
4,XGBoost,1.000000,1.000000,1.000000,1.000000,0.875469,0.875469,0.873740,0.874284


* It is also over-fitted

### Extracting the important features

In [ ]:
best_xgb_model.feature_importances_

array([0.00408087, 0.00487313, 0.02100447, 0.04538264, 0.01060886,
       0.00699862, 0.00889624, 0.0147527 , 0.0052189 , 0.00640659,
       0.00480575, 0.01083328, 0.00612646, 0.01140627, 0.00753237,
       0.01317193, 0.00468756, 0.01694987, 0.00630965, 0.00554302,
       0.00549452, 0.0099254 , 0.09896749, 0.3801331 , 0.07668985,
       0.13505888, 0.00354781, 0.00364734, 0.00381278, 0.00439561,
       0.00824566, 0.03941387, 0.00536041, 0.00173649, 0.00292472,
       0.00505691], dtype=float32)

In [ ]:
importances = best_xgb_model.feature_importances_
indices = np.argsort(importances)
print(indices)

[33 34 26 27 28  0 29 16 10  1 35  8 32 20 19 12 18  9  5 14 30  6 21  4
 11 13 15  7 17  2 31  3 24 22 25 23]


In [ ]:
indices = np.argsort(importances)[::-1] # np.argsort returns the indices that would sort an array.
pd.DataFrame([X_train.columns[indices], np.sort(importances)[::-1]])

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,outside.sideline_1.0,outside.baseline_1.0,outside.sideline_0.0,outside.baseline_0.0,net.clearance,previous.hitpoint_V,speed,previous.time.to.net,player.impact.depth,opponent.depth,...,serve,player.distance.from.center,opponent.distance.from.center,previous.hitpoint_F,rally,previous.hitpoint_B,same.side_1.0,same.side_0.0,gender_mens,server.is.impact.player_1.0
1,0.380133,0.135059,0.098967,0.07669,0.045383,0.039414,0.021004,0.01695,0.014753,0.013172,...,0.004873,0.004806,0.004688,0.004396,0.004081,0.003813,0.003647,0.003548,0.002925,0.001736


## Stacking:

In [ ]:
voting_clf = VotingClassifier(estimators=[('clf_dt', clf_dt), ('clf_rf', clf_rf), ('clf_adaboost', clf_adaboost)])

In [ ]:
voting_clf

VotingClassifier(estimators=[('clf_dt', DecisionTreeClassifier()),
                             ('clf_rf', RandomForestClassifier()),
                             ('clf_adaboost', AdaBoostClassifier())])

In [ ]:
voting_clf.fit(X_train, y_train)


VotingClassifier(estimators=[('clf_dt', DecisionTreeClassifier()),
                             ('clf_rf', RandomForestClassifier()),
                             ('clf_adaboost', AdaBoostClassifier())])

In [ ]:
y_pred_train = voting_clf.predict(X_train)
y_pred_train

array([1, 2, 0, ..., 1, 1, 0])

In [ ]:
y_pred_test = voting_clf.predict(X_test)
y_pred_test

array([1, 2, 2, ..., 2, 1, 1])

Performance Metric

In [ ]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "Stacking", scores)
scores

<ipython-input-63-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,1.000000,1.000000,1.000000,1.000000,0.797584,0.797584,0.796967,0.797268
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.861308,0.861308,0.859105,0.859650
2,GBM,0.919286,0.919286,0.918602,0.918517,0.866306,0.866306,0.863876,0.864530
3,Adaboost,0.849643,0.849643,0.847281,0.847952,0.836318,0.836318,0.833453,0.834513
4,XGBoost,1.000000,1.000000,1.000000,1.000000,0.875469,0.875469,0.873740,0.874284
5,Stacking,1.000000,1.000000,1.000000,1.000000,0.857143,0.857143,0.855144,0.855881


* After apply different classification models AdaBoost is well fit among all for this dataset.